In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
import os

# To import the self made functions you need to fill in the path were you 
# saved the functions instead of the word 'Code'
import sys
#sys.path.append('C:\\Users\\s158607\\Documents\\2019-2020\\Data challenge 3\\JBG060-Data-Challenge-3-master\\code')
sys.path.append('C:\\Users\\s158607\\PycharmProjects\\DataChallenge3\\Model 6\\code')

import utility
import load_files as lf
import flow_level_conversion as flc
import preprocessing as pre
import measurement_analysis as mea
import flow_model as model
import data_imputation as impu
import pred_to_rain as ptr

import warnings
warnings.filterwarnings('ignore')

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [2]:
# Fill the right directories for the data for Aa en Maas in here.
PATH = 'C:\\Users\\20174814\\Documents\\2019-2020\\Data challenge 3\\' # CHANGE (!)
PATH_MEASUREMENTS = PATH + "sewer_data\\data_pump\\RG8150"
PATH_RAIN_DATA = PATH + "sewer_data\\rain_timeseries"
PATH_SHAPE_FILES = PATH + "sewer_model\\aa-en-maas_sewer_shp"
PATH_RAIN_PREDICTION = PATH + "sewer_data\\rain_grid_prediction"

In [3]:
# LOADING NECESSARY DATA
# Measurements
flow_data, level_data = lf.get_measurements(PATH_MEASUREMENTS)
flow_data = pre.fill_flow(pre.clean_mes_data(flow_data))
level_data = pre.fill_level(pre.clean_mes_data(level_data))

# # Actual rain
rain_data = lf.get_rain(PATH_RAIN_DATA)

# Rain predicton
pred_days, rain_prediction = lf.get_rain_prediction(PATH_RAIN_PREDICTION, reduce_grid = True)

# Shape file data frames
area_data = lf.sdf(PATH_SHAPE_FILES).area_data

In [4]:
# creating the dwaas haas table
area_data.crs = {'init': 'epsg:28992'}
measure = mea.measurement_analysis(flow_data, level_data, rain_data, area_data = area_data, village_code = 'DRU')
dwaas_table = measure.compare_flow()
dwaas_table

,Name,Value,Rel. Value
0,Theoretical DWF (Q80),1756.063352,1.000000
1,Summer,3658.160068,2.083160
2,Winter,2576.862376,1.467409
3,Workday,2999.485208,1.708073
4,Weekend,3195.785694,1.819858
5,Average,3054.961432,1.739665


In [27]:
# creating the imputated flow
imputed_flow = impu.fill_flow(flow_data, level_data)
imputed_flow

C:\Users\20174814\Documents\2019-2020\Data challenge 3\JBG060-Data-Challenge-3-master\Utility\data_imputation.py:86: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  flow_values = flow_data.loc[same_level_timestamps]['Value']
C:\Users\20174814\Documents\2019-2020\Data challenge 3\JBG060-Data-Challenge-3-master\Utility\data_imputation.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  flow_data = flow_data), axis = 1)


0          789.470100
1          781.623800
2          788.569200
3          788.704500
4          789.410400
5          784.205400
6          778.151900
7          782.990600
8          781.058800
9          783.236800
10         787.941100
11         783.785500
12         788.164500
13         786.169700
14         785.905000
15         791.113900
16         787.861300
17         793.535500
18         780.900100
19         784.984500
20         784.238000
21         787.854100
22         784.290500
23         791.661900
24         788.677100
25         786.770300
26         789.513200
27         787.225100
28         784.494500
29         783.263900
              ...    
3305008    191.735800
3305009    184.527700
3305010    166.820400
3305011    148.522200
3305012    130.842200
3305013    106.863300
3305014     89.076600
3305015     71.372180
3305016     49.191600
3305017     27.238690
3305018      6.682343
3305019      0.764219
3305020      0.347216
3305021      0.000000
3305022   

In [6]:
# creating the prediction of the hourly flow
flow_model = model.flow_model(flow_data, level_data, (pred_days, rain_prediction))
cross_validation_score = flow_model.StochasticGradientDescent()
cross_validation_score

Train on 12255 samples, validate on 1362 samples
Epoch 1/400
12255/12255 [==============================] - 1s 42us/step - loss: 48195.1618 - val_loss: 31799.5561
Epoch 2/400
12255/12255 [==============================] - 0s 32us/step - loss: 43491.0905 - val_loss: 32709.2845
Epoch 3/400
12255/12255 [==============================] - 0s 27us/step - loss: 43328.5814 - val_loss: 32114.0792
Epoch 4/400
12255/12255 [==============================] - 0s 28us/step - loss: 42710.1407 - val_loss: 31589.1920
Epoch 5/400
12255/12255 [==============================] - 0s 30us/step - loss: 42275.4146 - val_loss: 31447.7035
Epoch 6/400
12255/12255 [==============================] - 0s 32us/step - loss: 41900.2681 - val_loss: 31445.5272
Epoch 7/400
12255/12255 [==============================] - 0s 31us/step - loss: 41578.4451 - val_loss: 31391.5916
Epoch 8/400
12255/12255 [==============================] - 0s 30us/step - loss: 41284.8784 - val_loss: 31269.1087
Epoch 9/400
12255/12255 [==============

Epoch 72/400
12255/12255 [==============================] - 0s 23us/step - loss: 33700.7739 - val_loss: 26315.1928
Epoch 73/400
12255/12255 [==============================] - 0s 23us/step - loss: 33651.9715 - val_loss: 26285.5018
Epoch 74/400
12255/12255 [==============================] - 0s 23us/step - loss: 33604.3240 - val_loss: 26256.7831
Epoch 75/400
12255/12255 [==============================] - 0s 23us/step - loss: 33557.7886 - val_loss: 26229.0117
Epoch 76/400
12255/12255 [==============================] - 0s 23us/step - loss: 33512.3387 - val_loss: 26202.1608
Epoch 77/400
12255/12255 [==============================] - 0s 23us/step - loss: 33467.9570 - val_loss: 26176.2026
Epoch 78/400
12255/12255 [==============================] - 0s 23us/step - loss: 33424.6274 - val_loss: 26151.1111
Epoch 79/400
12255/12255 [==============================] - 0s 23us/step - loss: 33382.2846 - val_loss: 26126.8668
Epoch 80/400
12255/12255 [==============================] - 0s 24us/step - loss:

Epoch 143/400
12255/12255 [==============================] - 0s 24us/step - loss: 31951.9868 - val_loss: 25566.2250
Epoch 144/400
12255/12255 [==============================] - 0s 23us/step - loss: 31940.9052 - val_loss: 25565.5203
Epoch 145/400
12255/12255 [==============================] - 0s 23us/step - loss: 31929.9960 - val_loss: 25564.9301
Epoch 146/400
12255/12255 [==============================] - 0s 23us/step - loss: 31919.2685 - val_loss: 25564.4520
Epoch 147/400
12255/12255 [==============================] - 0s 23us/step - loss: 31908.7208 - val_loss: 25564.0850
Epoch 148/400
12255/12255 [==============================] - 0s 23us/step - loss: 31898.3283 - val_loss: 25563.8281
Epoch 149/400
12255/12255 [==============================] - 0s 23us/step - loss: 31888.0950 - val_loss: 25563.6746
Epoch 150/400
12255/12255 [==============================] - 0s 26us/step - loss: 31878.0273 - val_loss: 25563.6224
Epoch 151/400
12255/12255 [==============================] - 0s 25us/ste

12255/12255 [==============================] - 0s 33us/step - loss: 31209.0373 - val_loss: 25913.3474
Epoch 284/400
12255/12255 [==============================] - 0s 27us/step - loss: 31206.4506 - val_loss: 25916.6324
Epoch 285/400
12255/12255 [==============================] - 0s 28us/step - loss: 31203.8810 - val_loss: 25919.9125
Epoch 286/400
12255/12255 [==============================] - 0s 32us/step - loss: 31201.3348 - val_loss: 25923.1881
Epoch 287/400
12255/12255 [==============================] - 0s 33us/step - loss: 31198.8011 - val_loss: 25926.4627
Epoch 288/400
12255/12255 [==============================] - 0s 31us/step - loss: 31196.2967 - val_loss: 25929.7339
Epoch 289/400
12255/12255 [==============================] - 0s 30us/step - loss: 31193.7929 - val_loss: 25932.9982
Epoch 290/400
12255/12255 [==============================] - 0s 30us/step - loss: 31191.3157 - val_loss: 25936.2609
Epoch 291/400
12255/12255 [==============================] - 0s 34us/step - loss: 3118